# Experimentación métodos iterativos

In [2]:
import os
import numpy as np
import subprocess as sp

In [3]:
dir = "../data"

## Funciones auxiliares

In [4]:
def generar_sistema_edd(tam, sol, tipo):
    A = np.random.randint(1, 40, (tam, tam))
    suma = np.sum(np.abs(A), axis=1)
    A += np.diag(suma)

    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_edd_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

def generar_sistema_simetrico(tam, sol, tipo):
    A = np.random.randint(1, 40, (tam, tam))
    A = A.T @ A

    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_sim_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

def generar_sistema_triangular(tam, sol, tipo):
    A = np.tril(np.random.randint(1, 40, (tam, tam)))
    b = A @ sol

    if not os.path.exists(f"{dir}/{tipo}"):
        os.makedirs(f"{dir}/{tipo}")

    np.savetxt(f"{dir}/{tipo}/matriz_triang_{tam}.txt", np.hstack((A, b[:, np.newaxis])), "%d", " ", comments="", header=f"{tam}")

In [5]:
def correr_algoritmo(dataset, tipo, tam, metodo, iteraciones, tol):        
    proceso = sp.run(["../src/iterativo", f"{dir}/{dataset}/matriz_{tipo}_{tam}.txt", metodo, str(iteraciones), str(tol)], capture_output=True, text=True)
    proceso.check_returncode()

    return np.array(proceso.stdout.split(" "), dtype=np.float64)


In [6]:
def crear_archivos(n, low, high, tipo):
    for i in range(2, n + 1):
        tam = 2 ** i
        x = np.random.randint(low, high, tam)
        
        generar_sistema_edd(tam, x, tipo)
        generar_sistema_simetrico(tam, x, tipo)
        generar_sistema_triangular(tam, x, tipo)

        np.savetxt(f"{dir}/{tipo}/vector_x_{tam}.txt", x, "%d", " ")

## Generar instancias

In [7]:
n = 8

### Vector solucion chico

In [8]:
crear_archivos(n, 0, 10, "chico")

### Vector solucion mediano

In [9]:
crear_archivos(n, 50, 100, "mediano")

### Vector solucion grande

In [10]:
crear_archivos(n, 200, 300, "grande")

## Correr algoritmos

In [11]:
# res = []
# correr_algoritmo("chico", "LU", 1000, 1e-20)

-Error de aproximacion entre valor actual y final (por cantidad de iteraciones) para matrices de diferentes tamaños
-Tiempo final de computo por iteraciones y por tamaño de matrices
-LU tiempo de computo por tamaño de matriz. Error numerico (comparar con x real).
-Comparar LU con todos los metodos iterativos.

## Generación de Gráficos

### Error

In [12]:
def cargar_soluciones(tipo):
    for i in range(2, n + 1):
        tam = 2 ** i
        xs.append(np.genfromtxt(f"{dir}/{tipo}/vector_x_{tam}.txt"))

In [20]:
skipped = 0
datasets = ["chico", "mediano", "grande"]
algoritmos = ["J", "JS", "GS", "GSS"]
tipos = ["edd", "sim", "triang"]
max_iter = 10
step = 2
tolerancia = 1e-10
resultados = {}

for ds in datasets:
    for algoritmo in algoritmos:
        resultados[f"{algoritmo}"] = []
        for tipo in tipos:
            for i in range(2, n + 1):
                for j in range(1, max_iter + 1, step):
                    t = %timeit -r 5 -n 3 -o correr_algoritmo(ds, tipo, 2 ** i, algoritmo, j, tolerancia)
                    res = correr_algoritmo(ds, tipo, 2 ** i, algoritmo, j, tolerancia)
                    resultados[f"{algoritmo}"][f"{tipo}"][f"{2**i}"].append(
                        {
                         "niter": j,
                         "tiempo": t,
                         "res": res
                        })


for ds in datasets:
    resultados["LU"] = []
    for tipo in tipos:
        for i in range (2, n + 1):
            t = %timeit -r 5 -n 3 -o correr_algoritmo(ds, tipo, 2 ** i, "LU", 0, tolerancia)
            res = correr_algoritmo(ds, tipo, 2 ** i, "LU", 0, tolerancia)
            resultados["LU"]["tipo"]["tamanio"].append(
                {
                 "tiempo": t,
                 "res": res
                })
resultados

3.74 ms ± 1.88 ms per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.91 ms ± 267 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.87 ms ± 416 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
3.01 ms ± 569 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.17 ms ± 258 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.19 ms ± 285 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.76 ms ± 345 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.56 ms ± 335 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.68 ms ± 129 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.17 ms ± 393 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.33 ms ± 319 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.95 ms ± 218 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
3.06 ms ± 642 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.88 ms ± 114 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)
2.29 ms ± 522 µs per loop (mean ±

{'J': [{'tipo': 'edd',
   'tamanio': 4,
   'niter': 1,
   'tiempo': <TimeitResult : 1.52 ms ± 19.1 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)>,
   'res': array([400.666, 405.436, 397.648, 441.572])},
  {'tipo': 'edd',
   'tamanio': 4,
   'niter': 3,
   'tiempo': <TimeitResult : 1.56 ms ± 66 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)>,
   'res': array([283.952, 348.003, 292.518, 349.221])},
  {'tipo': 'edd',
   'tamanio': 4,
   'niter': 5,
   'tiempo': <TimeitResult : 1.53 ms ± 25.8 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)>,
   'res': array([237.844, 314.804, 248.349, 304.4  ])},
  {'tipo': 'edd',
   'tamanio': 4,
   'niter': 7,
   'tiempo': <TimeitResult : 1.56 ms ± 29 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)>,
   'res': array([217.376, 298.826, 228.212, 283.684])},
  {'tipo': 'edd',
   'tamanio': 4,
   'niter': 9,
   'tiempo': <TimeitResult : 1.55 ms ± 51.7 µs per loop (mean ± std. dev. of 5 runs, 3 loops each)>,
   'res': array([208

In [25]:
xs = []
print(resultados["LU"][-1])

{'tipo': {'triang'}, 'tamanio': {256}, 'tiempo': <IPython.core.magics.execution.TimeitResult object at 0x7ff5ba62b150>, 'res': array([   234.    ,    237.    ,    293.    ,    216.    ,    262.    ,
          201.    ,    211.    ,    218.    ,    297.    ,    294.    ,
          227.    ,    221.    ,    254.    ,    227.    ,    282.    ,
          243.    ,    204.    ,    237.    ,    221.    ,    270.    ,
          247.    ,    209.    ,    254.    ,    219.    ,    227.    ,
          251.    ,    291.    ,    231.    ,    287.    ,    238.    ,
          240.    ,    219.    ,    264.    ,    205.    ,    247.    ,
          297.    ,    237.    ,    261.    ,    241.    ,    280.    ,
          295.    ,    246.    ,    207.    ,    234.    ,    274.    ,
          241.    ,    202.    ,    289.    ,    223.    ,    285.    ,
          224.    ,    236.    ,    244.    ,    230.    ,    269.    ,
          259.    ,    237.    ,    284.    ,    276.    ,    209.    ,
         